In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")
from datetime import datetime

# from dotenv import load_dotenv
# load_dotenv()
import geopandas as gpd
import contextily as cx
import gtfs_kit as gk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from rasterio.plot import show
import seaborn as sns

from openbustools import standardfeeds

### Energy Use for KCM Network

In [ ]:
network_name = "kcm"
res_dir = Path("..","results","energy",network_name,"sensitivity","baseline")
res_dir = Path("..","ExtremeSSD","old_data_and_results","results","energy",network_name,"sensitivity","baseline")
epsg = 32148

file = open(res_dir / "trajectories_updated.pkl", "rb")
trajectories_updated = pickle.load(file)
file.close()

file = open(res_dir / "depot_locations.pkl", "rb")
depot_locations = pickle.load(file)
file.close()

file = open(res_dir / "cycles.pkl", "rb")
cycles = pickle.load(file)
file.close()

file = open(res_dir / "network_energy.pkl", "rb")
network_energy = pickle.load(file)
file.close()

file = open(res_dir / "network_charging.pkl", "rb")
network_charging = pickle.load(file)
file.close()

file = open(res_dir / "veh_status.pkl", "rb")
veh_status = pickle.load(file)
file.close()

file = open(res_dir / ".." / ".." / "network_sensitivity.pkl", "rb")
network_sensitivity = pickle.load(file)
file.close()

In [ ]:
network_sensitivity[network_sensitivity['file']=="baseline"]

In [ ]:
plot_df = network_energy.groupby("block_id").first()

# Drive cycle reported trip consumption
fig, axes = plt.subplots(1,1, figsize=(8,4))

sns.histplot(plot_df['block_consumption_kwh_mi'], kde=True, ax=axes)
axes.axvline(x=np.mean(plot_df['block_consumption_kwh_mi']), linestyle='dashed', color='red')
axes.text(x=np.mean(plot_df['block_consumption_kwh_mi']) + .1, y=145, s=f"Avg. {np.mean(plot_df['block_consumption_kwh_mi']):.3} kWh/mi")
axes.set_xlabel("Consumption (kWh/mi)")
axes.set_ylabel(f"Count ({len(plot_df):,} Total Blocks)")

fig.suptitle(f"Modeled BEB Block Consumption for Full KCM Network")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_block_consumption_distribution.png"))
plt.show()

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,4))

sns.ecdfplot(plot_df['block_net_energy_kwh'], ax=axes)
axes.axvline(x=466, linestyle='dashed', color='red')
axes.text(x=486, y=0.4, s="Design Vehicle (466 kWh)")
axes.set_xlabel("Total Block Energy (kWh)")
axes.set_ylabel("Proportion of Blocks Met")

fig.suptitle(f"Modeled BEB Block Energy Needs for Full KCM Network")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_block_energy_distribution.png"))
plt.show()

In [ ]:
# Depot locations
block_starts_df = network_energy.groupby('block_id').first()
block_starts_df = gpd.GeoDataFrame(block_starts_df, geometry=gpd.points_from_xy([t.x for t in block_starts_df['start_loc']], [t.y for t in block_starts_df['start_loc']])).set_crs(epsg)
depot_df = gpd.GeoDataFrame(depot_locations, geometry=gpd.points_from_xy(depot_locations['depot_x'], depot_locations['depot_y'])).set_crs(epsg)

fig, axes = plt.subplots(1,1, figsize=(6,6))
block_starts_df.plot(ax=axes, column='depot_id', markersize=10, cmap='tab20')
depot_df.plot(ax=axes, marker='x', markersize=100, color="blue", linewidth=3)
cx.add_basemap(ax=axes, crs=block_starts_df.crs.to_string(), source=cx.providers.CartoDB.Positron)
axes.set_xticks([])
axes.set_yticks([])

fig.suptitle("Clustered Depot Locations")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_depot_locations.png"))

### Charging for KCM Network

In [ ]:
# Charging rate for 10/95% of blocks (unmanaged)
network_sensitivity[(network_sensitivity['metric'].isin(["Charger Power 95% (kW)","Charger Power 10% (kW)"])) & (network_sensitivity['file']=="baseline")]

In [ ]:
# Minimum charging rate to cover block energy (managed)
print(f"{network_charging['min_charge_rate_managed'].iloc[0]} Avg. kW per-vehicle")
print(f"{network_charging['min_charge_rate_managed'].iloc[0] * veh_status['tot_veh_inactive'].max()} Peak kW")

In [ ]:
# Blocks meeting pullout under baseline
print(len(network_charging[network_charging['charge_time_min'] < network_charging['t_until_pullout_min']]) / len(network_charging))

In [ ]:
veh_status['t_hour_of_day'] = veh_status['t_min_of_day'] / 60

In [ ]:
sns.set_style("ticks")
fig, axes = plt.subplots(1,1, figsize=(8,5))
axes2 = plt.twinx()

sns.lineplot(veh_status, x='t_hour_of_day', y='tot_veh_active', ax=axes, color=sns.color_palette()[0], label="Active Vehicles")
sns.lineplot(veh_status, x='t_hour_of_day', y='tot_veh_inactive', ax=axes, color=sns.color_palette()[1], label="Inactive Vehicles")
axes.set_ylim(0,1500)
axes.set_xlim(0,24)
axes.set_xlabel("Time of Day (hours)")
axes.set_ylabel("Number of Vehicles")
axes.legend().remove()

sns.lineplot(veh_status, x='t_hour_of_day', y='tot_power', ax=axes2, color=sns.color_palette()[2], label="Unmanaged Power")
axes2.axhline(y=veh_status['tot_power'].max(), linestyle='dashed', color='red')
axes2.text(x=10, y=veh_status['tot_power'].max()-.1*veh_status['tot_power'].max(), s=f"Peak Demand: {veh_status['tot_power'].max()/1000:.1f} MW")
axes2.set_xlabel("Time of Day (hours)")
axes2.set_ylabel("Power Demand (kW)")
axes2.legend().remove()

lines, labels = axes.get_legend_handles_labels()
lines2, labels2 = axes2.get_legend_handles_labels()
axes2.legend(lines + lines2, labels + labels2, loc='best')

fig.suptitle("Vehicle Status by Time of Day")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","kcm_veh_status.png"))
sns.set_style('darkgrid')

In [ ]:
# min_bins = np.arange(0, 1440, 15)
# veh_status['t_min_of_day_bin'] = np.digitize(veh_status['t_min_of_day'], min_bins) * 15
# plot_df = veh_status.groupby('t_min_of_day_bin').sum()

# fig, axes = plt.subplots(1,1, figsize=(8,4))
# axes2 = plt.twinx()

# sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_veh_arriving', ax=axes, color=sns.color_palette()[0], label="Vehicles Arriving (15min)")
# sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_veh_departing', ax=axes, color=sns.color_palette()[1], label="Vehicles Departing (15min)")
# axes.set_xlabel("Time of Day (minutes)")
# axes.set_ylabel("Number of Vehicles")
# axes.legend().remove()

# # sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_energy_arriving', ax=axes2, color=sns.color_palette()[0], linestyle='dashed', label="Power Unmanaged")
# sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_energy_departing', ax=axes2, color=sns.color_palette()[1], linestyle='dashed', label="Power Unmanaged")
# axes2.set_xlabel("Time of Day (minutes)")
# axes2.set_ylabel("Energy Needs (kWh)")
# axes2.legend().remove()

# lines, labels = axes.get_legend_handles_labels()
# # lines2, labels2 = axes2.get_legend_handles_labels()
# axes2.legend(lines, labels, loc='best')

# fig.suptitle("Vehicle Availability and Needs by Time of Day")
# fig.tight_layout()
# plt.show()
# fig.savefig(Path("..","plots","kcm_block_pullout.png"))

### Validate Block Energy w/KCM Report

In [ ]:
# Report table
block_energy_summary = network_energy.groupby('block_id').first()[['block_dist_mi', 'block_consumption_kwh_mi', 'block_net_energy_kwh']]
block_energy_summary.describe().transpose()

In [ ]:
# # Load the daily summaries from KCM report
# summary_data = standardfeeds.clean_parametrix("../ExtremeSSD/old_data_and_results/data/bebdatafollowup/Viriciti_Energy_Reports-2023.csv")
# summary_data = summary_data[summary_data['DateTime'] >= datetime(2023, 12, 1)]
# summary_data['realtime_filename'] = summary_data['DateTime'].dt.strftime("%Y_%m_%d")
# summary_data = summary_data.groupby(['realtime_filename','vehicle_id','metric']).agg({'value':'mean'}).reset_index().sort_values(['realtime_filename', 'vehicle_id', 'metric'])

# # Load the most recent static feed
# static_path = Path("..","data","kcm_static","2023_09_27")
# static = gk.read_feed(static_path, dist_units='km')

# # Load realtime data from all BEB vehicle IDs in the KCM report
# realtime_path = Path("..","data","kcm_realtime","processed", "analysis")
# beb_ids = summary_data['vehicle_id'].unique()
# beb_dates = summary_data['realtime_filename'].unique()
# all_realtime_data = []
# for d in beb_dates:
#     realtime_data = pd.read_pickle(Path(f"../data/kcm_realtime/processed/analysis/{d}.pkl"))
#     realtime_data = realtime_data[realtime_data['vehicle_id'].isin(beb_ids)]
#     realtime_data['realtime_filename'] = realtime_data['realtime_filename'].str[:-4]
#     all_realtime_data.append(realtime_data)
# all_realtime_data = pd.concat(all_realtime_data).sort_values(['realtime_filename','vehicle_id','trip_id','locationtime'])

# # Map (day, vehicle_id) > trip_ids using the realtime data
# trip_id_lookup = all_realtime_data[['realtime_filename','vehicle_id','trip_id']].drop_duplicates().copy()
# # Map trip_id > (service_id, block_id) using the static data
# block_id_lookup = static.get_trips()[['service_id','block_id','trip_id']].drop_duplicates().copy()
# block_id_lookup = pd.merge(trip_id_lookup, block_id_lookup, on='trip_id')
# block_id_lookup = block_id_lookup[['realtime_filename','vehicle_id','service_id','block_id']].drop_duplicates().copy()
# # Join energy summaries to their block_ids; note there are days where the vehicle was tracked on multiple blocks in the realtime
# summary_data = pd.merge(summary_data, block_id_lookup, on=['realtime_filename','vehicle_id'])
# # Remove any records of 48XX which are 60' buses
# summary_data = summary_data[~summary_data['vehicle_id'].str.startswith("48")]
# # Get comparison metrics for each block
# summary_data_means = summary_data.groupby(['block_id','metric'], as_index=False).agg({'value': 'mean'}).pivot(index='block_id', columns='metric', values='value')
# summary_data_stds = summary_data.groupby(['block_id','metric'], as_index=False).agg({'value': 'std'}).pivot(index='block_id', columns='metric', values='value').mean()
# real_beb_comparison = pd.merge(summary_data_means, block_energy_summary, on='block_id')
# real_beb_comparison.head()

In [ ]:
# plot_df = real_beb_comparison
# plot_df['block_dist_adj_mi'] = plot_df['block_dist_mi'] * 1.0
# plot_df['block_consumption_adj_kwh_mi'] = plot_df['block_consumption_kwh_mi'] * 2.0
# plot_df['block_net_energy_adj_kwh'] = plot_df['block_consumption_adj_kwh_mi'] * plot_df['block_dist_adj_mi']

# fig, axes = plt.subplots(1,2, figsize=(8,4))
# sns.scatterplot(data=plot_df, x='block_net_energy_kwh', y='Energy used', ax=axes[0])
# axes[0].axline([0,0], [1,1], color='red', linestyle='--', alpha=.8)
# axes[0].axline([0, 0+summary_data_stds['Energy used']], [1, 1+summary_data_stds['Energy used']], color='red', alpha=.3)
# axes[0].axline([0, 0-summary_data_stds['Energy used']], [1, 1-summary_data_stds['Energy used']], color='red', alpha=.3)
# axes[0].set_xlim(0, 800)
# axes[0].set_ylim(0, 800)
# axes[0].set_xlabel("Modeled Energy (kWh)")
# axes[0].set_ylabel("Actual Energy (kWh)")
# axes[0].set_title("40ft Bus")

# sns.scatterplot(data=plot_df, x='block_net_energy_adj_kwh', y='Energy used', ax=axes[1])
# axes[1].axline([0,0], [1,1], color='red', linestyle='--', alpha=.8)
# axes[1].axline([0, 0+summary_data_stds['Energy used']], [1, 1+summary_data_stds['Energy used']], color='red', alpha=.3)
# axes[1].axline([0, 0-summary_data_stds['Energy used']], [1, 1-summary_data_stds['Energy used']], color='red', alpha=.3)
# axes[1].set_xlim(0, 800)
# axes[1].set_ylim(0, 800)
# axes[1].set_xlabel("Modeled Energy (kWh)")
# axes[1].set_ylabel("Actual Energy (kWh)")
# axes[1].set_title("Adjusted for Reported Consumption")
# fig.suptitle(f"Modeled vs. Actual Block Energy Consumption for KCM\n{len(plot_df)} Blocks")
# fig.tight_layout()
# fig.savefig(Path("..", "plots", "kcm_real_beb_comparison.png"))
# plt.show()

### KCM Sensitivity Analysis and Performance Metrics

In [ ]:
baseline_sensitivity = network_sensitivity[network_sensitivity['file']=="baseline"]

In [ ]:
plot_metrics = [
    'Avg. Block Consumption (kWh/mi)',
    'Avg. Trip Consumption (kWh/mi)',
]
plot_parameters = [
    'Acc./Dec. Factor',
    "Aux Power",
    "Deadhead Consumption",
    "Depot Density",
    "Door Open Time",
    "Passenger Load",
    "Temperature"
]

plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]

# Compare metrics across sensitivity parameters
fig, axes = plt.subplots(2, 1, figsize=(8,5))
axes = axes.flatten()
for i, metric in enumerate(plot_df['metric'].unique()):
    # Filter to metric for subplot, plot the ranges
    ranges = plot_df[plot_df['metric']==metric].groupby('sensitivity_parameter')['value'].agg(['min', 'max'])
    for j, (param, row) in enumerate(ranges.iterrows()):
        axes[i].plot([row['min'], row['max']], [j, j], color=sns.color_palette()[0], linewidth=3)
        axes[i].set_yticks(range(len(ranges)))
        axes[i].set_yticklabels(ranges.index)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Sensitivity Parameter')

    axes[i].vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
    axes[i].set_ylabel("")
    axes[i].set_xlabel(metric)
    axes[i].set_xlim(0.5, 6.0)

fig.suptitle("Sensitivity of Energy Consumption in Full KCM Network")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_sensitivity_consumption.png"))

In [ ]:
plot_metrics = [
    'Avg. Block Energy (kWh)',
    'Battery Capacity 10% (kWh)',
    'Battery Capacity 95% (kWh)'
]
plot_parameters = [
    'Acc./Dec. Factor',
    "Aux Power",
    "Deadhead Consumption",
    "Depot Density",
    "Door Open Time",
    "Passenger Load",
    "Temperature"
]

plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]
plot_df

# Compare metrics across sensitivity parameters
fig, axes = plt.subplots(3, 1, figsize=(8,7.5))
axes = axes.flatten()
for i, metric in enumerate(plot_df['metric'].unique()):
    # Filter to metric for subplot, plot the ranges
    ranges = plot_df[plot_df['metric']==metric].groupby('sensitivity_parameter')['value'].agg(['min', 'max'])
    for j, (param, row) in enumerate(ranges.iterrows()):
        axes[i].plot([row['min'], row['max']], [j, j], color=sns.color_palette()[0], linewidth=3)
        axes[i].set_yticks(range(len(ranges)))
        axes[i].set_yticklabels(ranges.index)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Sensitivity Parameter')
    axes[i].vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
    axes[i].set_ylabel("")
    axes[i].set_xlabel(metric)

fig.suptitle("Sensitivity of Block Energy Needs in Full KCM Network")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_sensitivity_energy.png"))

### All Networks Sensitivity Analysis and Performance Metrics

In [ ]:
cleaned_sources = pd.read_csv("../data/cleaned_sources.csv")

all_network_res = []
all_network_provider = []
all_network_available = []
all_network_energy = []
for i,row in cleaned_sources.iterrows():
    try:
        network_sensitivity = pd.read_pickle(Path("..","ExtremeSSD","old_data_and_results","results","energy",row['uuid'],"network_sensitivity.pkl"))
        all_network_res.append(network_sensitivity)
        all_network_provider.append(row['provider'])
        veh_status = pd.read_pickle(Path("..","ExtremeSSD","old_data_and_results","results","energy",row['uuid'],"sensitivity","baseline","veh_status.pkl"))
        all_network_available.append(veh_status['tot_veh_inactive'].sum() / 60)
        block_coverage = pd.read_pickle(Path("..","ExtremeSSD","old_data_and_results","results","energy",row['uuid'],"sensitivity","baseline","network_charging.pkl"))
        all_network_energy.append(block_coverage.groupby('block_id').first()['block_net_energy_kwh'].sum())
    except Exception as e:
        continue

all_network_avg_charge = pd.DataFrame({
    "Agency": all_network_provider,
    "Charge Availability (veh-hr)": all_network_available,
    "Daily Energy (kWh)": all_network_energy,
    "Avg. Power (kW)": [x / y for x,y in zip(all_network_energy, all_network_available)]
})

all_network_res = pd.concat(all_network_res)
# Throw the ones w/o block IDs
all_network_res = all_network_res[~all_network_res['provider'].isin([
    "Metro St. Louis",
    "Mountain View Transportation Management Association (MVgo)",
    "Massachusetts Bay Transportation Authority (MBTA)",
    "TransLink Sunbus Cairns",
    "Roma Servizi per la Mobility",
    "Big Blue Bus",
    "Valley Metro"
])]
print(all_network_res['provider'].nunique())
print(all_network_res['metric'].unique())
print(all_network_res['sensitivity_parameter'].unique())


In [ ]:
# 10% and 95% battery capacity needs
plot_metrics = [
    'Battery Capacity 10% (kWh)',
    'Battery Capacity 95% (kWh)',
]
sensitivity_metrics = [
    'Aux Power'
]
plot_df = all_network_res[all_network_res['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
plot_df = plot_df[plot_df['sensitivity_parameter'].isin(sensitivity_metrics)]
plot_df['metric'] = plot_df['metric'].replace({
    'Battery Capacity 10% (kWh)': '10% Block Electrification',
    'Battery Capacity 95% (kWh)': '95% Block Electrification'
})

fig, axes = plt.subplots(1, 1, figsize=(8,8))

# Filter to metric for subplot, plot the ranges
ranges_10 = plot_df[plot_df['metric']=='10% Block Electrification'].groupby(['provider','sensitivity_parameter','metric'])['value'].agg(['min', 'max']).reset_index()
ranges_95 = plot_df[plot_df['metric']=='95% Block Electrification'].groupby(['provider','sensitivity_parameter','metric'])['value'].agg(['min', 'max']).reset_index()
ranges_10 = ranges_10.sort_values('min', ascending=False)
ranges_95 = ranges_95.reindex(ranges_10.index)

# Filter to metric for subplot, plot the ranges
for j, (metric_10, metric_95) in enumerate(zip(ranges_10.iterrows(), ranges_95.iterrows())):
    row_10 = metric_10[1]
    row_95 = metric_95[1]
    axes.plot([row_10['min'], row_10['max']], [j, j], color=sns.color_palette()[0], linewidth=3, label='10% Block Electrification')
    axes.plot([row_95['min'], row_95['max']], [j-.25, j-.25], color=sns.color_palette()[1], linewidth=3, label='95% Block Electrification')
    axes.set_yticks(range(len(ranges_10)))
    axes.set_yticklabels([x for x in ranges_10['provider']])

axes.set_ylabel("")
axes.set_xlabel("Battery Capacity Sensitivity to Aux Load (kWh)")
axes.tick_params(axis='y', rotation=30, labelsize=8)

handles, labels = axes.get_legend_handles_labels()
handles = handles[:2]
labels = labels[:2]
axes.legend(handles, labels, loc='upper right')
fig.suptitle("Battery Capacity Required for Fleet Electrification")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","international_capacity.png"))

In [ ]:
# 10% and 95% charging power needs
plot_metrics = [
    'Charger Power 10% (kW)',
    'Charger Power 95% (kW)',
]
sensitivity_metrics = [
    'Aux Power'
]
plot_df = all_network_res[all_network_res['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
plot_df = plot_df[plot_df['sensitivity_parameter'].isin(sensitivity_metrics)]
plot_df['metric'] = plot_df['metric'].replace({
    'Charger Power 10% (kW)': '10% Block Electrification',
    'Charger Power 95% (kW)': '95% Block Electrification'
})

fig, axes = plt.subplots(1, 1, figsize=(8,8))

# Filter to metric for subplot, plot the ranges
ranges_10 = plot_df[plot_df['metric']=='10% Block Electrification'].groupby(['provider','sensitivity_parameter','metric'])['value'].agg(['min', 'max']).reset_index()
ranges_95 = plot_df[plot_df['metric']=='95% Block Electrification'].groupby(['provider','sensitivity_parameter','metric'])['value'].agg(['min', 'max']).reset_index()
ranges_10 = ranges_10.sort_values('min', ascending=False)
ranges_95 = ranges_95.reindex(ranges_10.index)

# Filter to metric for subplot, plot the ranges
for j, (metric_10, metric_95) in enumerate(zip(ranges_10.iterrows(), ranges_95.iterrows())):
    row_10 = metric_10[1]
    row_95 = metric_95[1]
    axes.plot([row_10['min'], row_10['max']], [j, j], color=sns.color_palette()[0], linewidth=3, label='10% Block Electrification')
    axes.plot([row_95['min'], row_95['max']], [j-.25, j-.25], color=sns.color_palette()[1], linewidth=3, label='95% Block Electrification')
    axes.set_yticks(range(len(ranges_10)))
    axes.set_yticklabels([x for x in ranges_10['provider']])

axes.set_xlim(0, 350)
axes.set_ylabel("")
axes.set_xlabel("Plug Power Sensitivity to Aux Load (kW)")
axes.tick_params(axis='y', rotation=30, labelsize=8)

handles, labels = axes.get_legend_handles_labels()
handles = handles[:2]
labels = labels[:2]
axes.legend(handles, labels, loc='upper right')
fig.suptitle("Plug Power Required to Meet Block Pullouts")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","international_power.png"))

In [ ]:
baseline = all_network_res[all_network_res['file']=="baseline"]
network_compare_df = baseline.groupby(['provider', 'metric'], as_index=False).first()
network_compare_df = network_compare_df.pivot(index='provider', columns='metric', values='value').reset_index()
network_compare_df = network_compare_df[network_compare_df['Avg. Block Distance (mi)'] < 400]
network_compare_df = pd.merge(network_compare_df, all_network_avg_charge, left_on='provider', right_on='Agency')
network_compare_df

In [ ]:
network_compare_df.drop(columns=['provider', 'Agency']).corr()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8,6))

sns.scatterplot(network_compare_df, x='Avg. Block Distance (mi)', y='Avg. Power (kW)', hue='provider', ax=axes)

axes.legend().remove()
fig.suptitle(f"Relationship Between Block Distance and Average Power")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","international_block_distance.png"))

In [ ]:
network_compare_df['Service (veh-mi)'] = network_compare_df['Number of Blocks'] * network_compare_df['Avg. Block Distance (mi)']

In [ ]:
df = network_compare_df[network_compare_df['Charge Availability (veh-hr)']<10000]
sns.pairplot(df[['Avg. Block Distance (mi)', 'Charge Availability (veh-hr)', 'Daily Energy (kWh)', 'Avg. Power (kW)', 'Number of Blocks', 'Service (veh-mi)']], corner=True)
plt.show()